# 10,000 Samples with Old Dates

Since most of the data from GNM is static anyway, use BBG CPR data to extend the possible dates of my samples to pre-crisis (hopefully).

I'll save time (and my BBG data allowance) by just sampling pools for which I already have BBG CPR data.

In [1]:
import os
os.chdir("/home/charles/src/LondonMirror/Prepayments/")

import prepayments as pp
import numpy as np
import pandas as pd
import ggplot as gg
import datetime

pm = pp.models.PoolModel()

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


Do we already have an index on `cusip`?

In [3]:
pm.data_pool.h5file.root.pools.indexedcolpathnames

['cusip', 'pool_number']

Yes.

In [4]:
import cPickle as pickle

In [5]:
sample_cusips = pickle.load(open("/media/gdaa/Charles/prepayment/sample_cusips.pickle","r"))

In [6]:
len(sample_cusips)

29598

In [7]:
pm.data_for_pn_random_dt(pm.data_pool.pool_number_for_cusip(sample_cusips[40]))

{'burnout': 19797.453819426984,
 'cato': -0.036699999999999733,
 'dt': Timestamp('2013-12-01 00:00:00'),
 'hpa': -0.017199305911684414,
 'incentive': 317.12664436413252,
 'lockin': 0.9295274123635261,
 'next_month_cpr': 0.070000000000000007,
 'seasonality': 12,
 'upfront_mip': 10.256410256410255}

In [8]:
data = []
for i,x in enumerate(np.random.choice(sample_cusips,10000,replace=False)):
    try:
        data.append(pm.data_for_pn_random_dt(pm.data_pool.pool_number_for_cusip(x)))
    except e:
        print "#{}: {} failed. {}".format(i,x, e)
#pickle.dump(data, open("/media/gdaa/Charles/prepayment/sample_data2.pickle", "wb"))

KeyError: Timestamp('1963-01-31 00:00:00')

In [9]:
%debug

> /home/charles/src/LondonMirror/Prepayments/pandas/index.pyx(585)pandas.index.DatetimeEngine.get_loc (pandas/index.c:10996)()

ipdb> up
> /usr/local/lib/python2.7/dist-packages/pandas/indexes/base.py(2106)get_loc()
   2104                 return self._engine.get_loc(key)
   2105             except KeyError:
-> 2106                 return self._engine.get_loc(self._maybe_cast_indexer(key))
   2107 
   2108         indexer = self.get_indexer([key], method=method, tolerance=tolerance)

ipdb> up
> /usr/local/lib/python2.7/dist-packages/pandas/tseries/index.py(1402)get_loc()
   1400             # needed to localize naive datetimes
   1401             key = Timestamp(key, tz=self.tz)
-> 1402             return Index.get_loc(self, key, method, tolerance)
   1403 
   1404         if isinstance(key, time):

ipdb> up
> /usr/local/lib/python2.7/dist-packages/pandas/core/generic.py(1779)xs()
   1777                                                       drop_level=drop_level)
   1778         else:

In [13]:
pm.data_for_pn_random_dt('216961')

{'burnout': 227483.77607704044,
 'cato': 0.53540000000000099,
 'dt': Timestamp('2008-05-01 00:00:00'),
 'hpa': 0.43490031476094471,
 'incentive': 3339.9926024536999,
 'lockin': 0.62888971083847778,
 'next_month_cpr': 0.0,
 'seasonality': 5,
 'upfront_mip': 0}

In [ ]:
jim = pd.DataFrame.from_records(data, index="dt")

In [ ]:
jim = jim.where(jim.next_month_cpr >= 0).dropna()

In [ ]:
jim.to_csv("/media/gdaa/Charles/prepayment/sample_data3.csv")
jim.to_csv("/data/prepayments/samples3.csv")

In [ ]:
gg.ggplot(jim, gg.aes(x='incentive')) + gg.geom_histogram(bins=50)

In [ ]:
'36202A5Q6'